Real Time Object Detection using YOLOv8 and OpenCV

In [ ]:
import numpy as numpy
import cv2
from ultralytics import YOLO
import time

In [ ]:
model = YOLO('yolov8n.pt')

# Model Configuration
model.conf = 0.25  # Confidence threshold
model.iou = 0.7    # IoU threshold for NMS

In [ ]:
def run_camera_check():
    # Camera Check for Video
    print("Testing camera...")
    
    # Try to open camera
    cap = cv2.VideoCapture(0)  # 0 = default camera
    
    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera")
        print("Trying camera index 1...")
        cap = cv2.VideoCapture(1)
        
        if not cap.isOpened():
            print("No camera found")
            return
    
    print("Camera opened successfully!")
    print("Camera info:")
    print(f"  > Width: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}")
    print(f"  > Height: {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
    print(f"  > FPS: {int(cap.get(cv2.CAP_PROP_FPS))}")
    print("\nShowing live feed - Press 'q' to quit")
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame")
            break
        
        frame_count += 1
        cv2.putText(frame, f"Frame: {frame_count}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'q' to quit", (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Camera Test', frame)
        
        # Check for 'q' key press
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print(f"\nTest completed! Captured {frame_count} frames")
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    print("- Camera released -")
    
run_camera_check()

In [ ]:
def create_label_box(results, frame, detection_count):         
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0]) # Bounding box coordinates                
                
                # Extract confidence and class
                conf = box.conf[0]
                cls = int(box.cls[0])
                class_name = model.names[cls]
                label = f"{class_name} {conf:.2f}"
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Draw label background
                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), 
                            (x1 + label_size[0], y1), (0, 255, 0), -1)
                
                # Draw label text
                cv2.putText(frame, label, (x1, y1 - 5), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                
                detection_count += 1
    return detection_count

In [ ]:
def add_performance_info(frame, fps, detection_count, fps_counter, start_time):
    elapsed_time = time.time() - start_time
    fps = fps_counter / elapsed_time if elapsed_time > 0 else 0
    
    # Add performance info overlay
    info_text = f"FPS: {fps:.1f} | Detections: {detection_count}"
    cv2.putText(frame, info_text, (10, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Add instructions
    cv2.putText(frame, "Press: 'q'-Quit, 's'-Save, 'p'-Pause", (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

In [ ]:
def handle_key_presses(paused, frame):
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Stopping detection...")
        return True, paused
    elif key == ord('s'):
        # Save screenshot
        filename = f"detection_screenshot_{int(time.time())}.jpg"
        cv2.imwrite(filename, frame)
        print(f"Screenshot saved: {filename}")
    elif key == ord('p'):
        # Toggle pause
        paused = not paused
        if paused:
            print("Detection paused")
        else:
            print("Detection resumed")
    return False, paused

In [ ]:
def run_yolov8_detection():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    print("Camera initialized successfully!")
    print("Starting real-time object detection...")
    print("Press 'q' to quit, 's' to save screenshot, 'p' to pause")
    
    # Metrics to track
    fps_counter = 0
    start_time = time.time()
    paused = False
    detection_count = 0

    try:
        while True:
            # In each iteration, one frame is captured and processed by the model
            detection_count = 0 # reset detection count for each frame
            if not paused:
                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame")
                    break

                results = model(frame, verbose=False)
                detection_count = create_label_box(results, frame, detection_count)

            # Calculate FPS
            fps_counter += 1
            add_performance_info(frame, 0, detection_count, fps_counter, start_time)

            # Display frame
            cv2.imshow('YOLOv8 Real-Time Object Detection', frame)

            # Handle key presses
            quitOpted, paused = handle_key_presses(paused, frame)
            if quitOpted:
                break
            
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()

        # Final statistics
        total_time = time.time() - start_time
        avg_fps = fps_counter / total_time if total_time > 0 else 0

        print(f"\nDetection Statistics:")
        print(f"   > Total frames processed: {fps_counter}")
        print(f"   > Total time: {total_time:.2f} seconds")
        print(f"   > Average FPS: {avg_fps:.2f}")
        print("- Object detection completed!")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
run_yolov8_detection()